## POS tagging using  vanila viterbi & modified Viterbi

#### Note: Please run the whole note book 

In [1]:
#Importing libraries
import nltk
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import random
import seaborn
from collections import Counter

pd.set_option('display.max_columns',5400)
pd.set_option('display.max_rows',5400)

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
nltk_data[:10]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')],
 [('Rudolph', 'NOUN'),
  ('Agnew', 'NOUN'),
  (',', '.'),
  ('55', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  ('and', 'CONJ'),
  ('former', 'ADJ'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Consolidated', 'NOUN'),
  ('Gold', 'NOUN'),
  ('Fields', 'NOUN'),
  ('PLC', 'NOUN'),
  (',', '.'),
  ('was', 'VERB'),
  ('named', 'VERB'),
  ('*-1', 'X'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
 

#### All words are tagged. 
#### words are theirs taggs are inside a tuple
#### A sentence is represented as a list which congtains tuple of all words with taggs
#### The whole Treebank is inside a list
#### '.' can be considered as start. i.e. P(tag|start) = P(tag|.)

In [3]:
len(nltk_data)

3914

In [4]:
#spliting into train_test

random.seed(100)
train_data,test_data=train_test_split(nltk_data,test_size=0.05,random_state=50)
print('Length of train data = ',len(train_data))
print('Length of test data = ',len(test_data))

Length of train data =  3718
Length of test data =  196


In [5]:
#creating a list of tagged word

train_tagd=[tw for sent in train_data for tw in sent]
test_tagd=[tw for sent in test_data for tw in sent]
print(len(train_tagd))
print(len(test_tagd))

95421
5255


In [6]:
train_tagd[0:15] #(word,POS tag)

[('The', 'DET'),
 ('problem', 'NOUN'),
 ('involves', 'VERB'),
 ('the', 'DET'),
 ('motion', 'NOUN'),
 ('of', 'ADP'),
 ('small', 'ADJ'),
 ('magnetic', 'ADJ'),
 ('fields', 'NOUN'),
 ('within', 'ADP'),
 ('superconductor', 'NOUN'),
 ('crystals', 'NOUN'),
 (',', '.'),
 ('*', 'X'),
 ('limiting', 'VERB')]

In [7]:
# creating a vocabulary by extracting unique tokens(words,symbols,numbers) and unique POS tags

#tokens
V=set([i[0] for i in train_tagd])

#POS tags
T = set([i[1] for i in train_tagd])

print('number of unique token = ',len(V))
print('number of unique POS tags = ', len(T))


number of unique token =  12059
number of unique POS tags =  12


In [8]:
# unique pos tags are
T

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

In [9]:
# Creating parameters for HMM model

# Emission Probability

def w_g_t ( word,tag, train_d=train_tagd):
    onlyt=[wt for wt in train_d if wt[1]==tag ]
    w=[w for w in onlyt if w[0]==word]
    emission = len(w)/len(onlyt)
    
    return emission

#Transition probability

def t2_g_t1(t2,t1,train_d=train_tagd):
    tags=[i[1] for i in train_d]
    t1_c=[t for t in tags if t == t1]
    t2_t1_c=0
    for i in range(len(tags)-1):
        if tags[i]==t1 and tags[i+1] ==t2 :
            t2_t1_c+=1
            
    transition=t2_t1_c/len(t1_c)
            
    return (transition)


        
    
    

In [10]:
# creating a matrix to represent the relationship P(t2|t1) TxT MATRIX

tags_matrix=np.zeros((len(T),len(T)),dtype='float32')
tags_matrix

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [11]:
for i,t1 in enumerate(list(T)):
    for j,t2 in enumerate(list(T)):
        tags_matrix[i,j]=t2_g_t1(t2,t1)
tags_matrix

array([[1.68022253e-02, 7.02054799e-02, 1.04880139e-01, 8.34760256e-03,
        1.39126717e-03, 6.15368150e-02, 3.21810782e-01, 3.94905806e-02,
        1.41267125e-02, 8.56164377e-04, 3.49957198e-02, 3.25556517e-01],
       [2.29973178e-02, 8.04906059e-03, 7.28248358e-02, 4.86776531e-01,
        1.22652361e-02, 6.89919526e-03, 2.08892301e-01, 4.10118811e-02,
        3.37293968e-02, 4.98275179e-03, 9.19892713e-02, 9.58221499e-03],
       [7.79306889e-02, 6.63239916e-04, 6.58265650e-02, 1.19383186e-02,
        1.02802189e-02, 2.02288181e-02, 6.99220717e-01, 6.66556135e-02,
        4.97429958e-03, 1.65809989e-02, 2.07262486e-02, 4.97429958e-03],
       [9.15531367e-02, 3.56558971e-02, 6.53950945e-02, 1.69015184e-01,
        3.11405212e-02, 2.30439864e-02, 1.10782407e-01, 3.47995311e-02,
        8.10432062e-02, 5.29388851e-03, 2.17983648e-01, 1.34293497e-01],
       [2.00856104e-02, 1.74514316e-02, 8.29766244e-02, 4.03029293e-01,
        1.97563390e-03, 5.82811981e-02, 2.46954232e-01, 4.34

In [12]:
# creating a DataFrame
tags_df= pd.DataFrame(tags_matrix,columns=list(T),index=list(T))
tags_df

,ADP,PRON,ADJ,VERB,PRT,NUM,NOUN,.,ADV,CONJ,X,DET
ADP,0.016802,0.070205,0.104880,0.008348,0.001391,0.061537,0.321811,0.039491,0.014127,0.000856,0.034996,0.325557
PRON,0.022997,0.008049,0.072825,0.486777,0.012265,0.006899,0.208892,0.041012,0.033729,0.004983,0.091989,0.009582
ADJ,0.077931,0.000663,0.065827,0.011938,0.010280,0.020229,0.699221,0.066656,0.004974,0.016581,0.020726,0.004974
VERB,0.091553,0.035656,0.065395,0.169015,0.031141,0.023044,0.110782,0.034800,0.081043,0.005294,0.217984,0.134293
PRT,0.020086,0.017451,0.082977,0.403029,0.001976,0.058281,0.246954,0.043464,0.009549,0.001646,0.014159,0.100428
NUM,0.034832,0.001489,0.032450,0.017267,0.027985,0.185770,0.351295,0.118190,0.002679,0.013992,0.211075,0.002977
NOUN,0.176853,0.004642,0.012027,0.146330,0.043537,0.009468,0.264037,0.241373,0.017327,0.042477,0.028878,0.013050
.,0.090836,0.065459,0.045104,0.088594,0.002421,0.080882,0.222920,0.093257,0.052188,0.057658,0.027260,0.173332
ADV,0.119587,0.014657,0.128914,0.345769,0.014657,0.031646,0.031646,0.134910,0.079614,0.006995,0.022985,0.068621
CONJ,0.053296,0.061711,0.117812,0.154745,0.004675,0.041608,0.351566,0.034596,0.052828,0.000468,0.007948,0.118747


In [13]:
# CREATING THE HMM-VITERBI ALGORITHM ( VANILA VITERBI FORM)

def h_viterbi(words,train_bag=train_tagd):
    pos=[]
    T=list(set([i[1] for i in train_bag]))
    
    for index, word in enumerate(words):
        state_p=[]
        for tag in T:
            if index ==0:
                transition_p=tags_df.loc['.',tag]
            else:
                transition_p = tags_df.loc[pos[-1],tag]
                
            #emission probability
            emission_p = w_g_t(words[index],tag)
            state=transition_p * emission_p
            state_p.append(state)
            
        max_p=max(state_p) #evaluating maximum likelyhood
        
        w_pos=T[state_p.index(max_p)]
        pos.append(w_pos)
    
    return list(zip(words,pos))

In [14]:
#creating tokens 

test_words=[i[0] for i in test_tagd]
test_words[:10]

['Editorials',
 'in',
 'the',
 'Greenville',
 'newspaper',
 'allowed',
 'that',
 'Mrs.',
 'Yeargin',
 'was']

In [15]:
test_out=h_viterbi(test_words)
test_out[:25]

[('Editorials', 'ADP'),
 ('in', 'ADP'),
 ('the', 'DET'),
 ('Greenville', 'NOUN'),
 ('newspaper', 'NOUN'),
 ('allowed', 'VERB'),
 ('that', 'ADP'),
 ('Mrs.', 'NOUN'),
 ('Yeargin', 'NOUN'),
 ('was', 'VERB'),
 ('wrong', 'ADJ'),
 (',', '.'),
 ('but', 'CONJ'),
 ('also', 'ADV'),
 ('said', 'VERB'),
 ('0', 'X'),
 ('the', 'DET'),
 ('case', 'NOUN'),
 ('showed', 'VERB'),
 ('how', 'ADV'),
 ('testing', 'NOUN'),
 ('was', 'VERB'),
 ('being', 'VERB'),
 ('overused', 'ADP'),
 ('*-2', 'X'),
 ('*T*-1', 'X'),
 ('.', '.'),
 ('--', '.'),
 ('And', 'CONJ'),
 ('the', 'DET'),
 ('USIA', 'NOUN'),
 ('said', 'VERB'),
 ('that', 'ADP'),
 ('all', 'DET'),
 ('of', 'ADP'),
 ('us', 'PRON'),
 ('could', 'VERB'),
 ('take', 'VERB'),
 ('extensive', 'ADJ'),
 ('notes', 'NOUN'),
 ('.', '.'),
 ('Meanwhile', 'ADV'),
 (',', '.'),
 ('many', 'ADJ'),
 ('market', 'NOUN'),
 ('watchers', 'NOUN'),
 ('say', 'VERB'),
 ('0', 'X'),
 ('recent', 'ADJ'),
 ('dividend', 'NOUN'),
 ('trends', 'NOUN'),
 ('raise', 'VERB'),
 ('another', 'DET'),
 ('warning

In [16]:
#checking accuracy using base HMM & VITERBI algorithm

chk_tags = [i for i, j in zip(test_out, test_tagd) if i == j] 

accuracy = len(chk_tags)/len(test_out)
accuracy

0.9042816365366317

## Laplace Smoothing to deal with unknown words
### So when an unknown word appear in the test data 
#### there are chances that the transition probability may be zero
#### the emission probability will definitely be zero
#### i.e. P(T2|T1) = 0 , if we did not have the sequence T1 -> T2 in the training data. As all P(T2|T1) are non zeros, we do not need to modify the transition probability.
#### P(W|T) = 0 , as we do not have the word in out training data
#### here T - tag of unknow word W 

#### Emission Probability

#### P(W|T) = ((# W as T)+K ) / (#T + K* #words )

#### WHERE,
#### #words - total number of words vailable in the vocabulary
#### K - a value bw=etween 0-1 acts as a weight given to unknown values
#### (# W as T) - nuber of time W is tagged as T

In [17]:
# modified Emission Probability

def w_g_t_md( word,tag,k=1,train_d=train_tagd):
    T=list(set([i[1] for i in train_d]))
    wtr=list(set([i[0] for i in train_d]))
    onlyt=[wt for wt in train_d if wt[1]==tag ]
    w=[w for w in onlyt if w[0]==word]
    
    emission = (len(w)+k)/(len(onlyt)+k*len(wtr))
    return emission
    
    

In [22]:
# MODIFIED HMM-VITERBI ALGORITHM ( BASE FORM) according to Laplace smoothing

def h_viterbi_lp(words,train_bag =train_tagd):
    pos=[]
    T=list(set([i[1] for i in train_bag]))
    
    for index, word in enumerate(words):
        state_p=[]
        for tag in T:
            if index ==0:
                transition_p=tags_df.loc['.',tag]
            else:
                transition_p = tags_df.loc[pos[-1],tag]
                
            #emission probability
            emission_p = w_g_t_md(words[index],tag)
            state=transition_p * emission_p
            state_p.append(state)
            
        max_p=max(state_p) #evaluating maximum likelyhood
        
        w_pos=T[state_p.index(max_p)]
        pos.append(w_pos)
    
    return list(zip(words,pos))

In [ ]:
#checking accuracy
test_lp = h_viterbi_lp(test_words)
chk_tags = [i for i, j in zip(test_lp, test_tagd) if i == j] 

accuracy_lp = len(chk_tags)/len(test_lp)
accuracy_lp

In [ ]:
# incorrect tag cases for base 
incorrect_tagged_cases = [[test_tagd[i-1],j,test_tagd[i+1]] for i, j in enumerate(zip(test_out, test_tagd)) if j[0]!=j[1]]

In [ ]:
# incorrect tag cases for lp
incorrect_tagged_cases_lp = [[test_tagd[i-1],j,test_tagd[i+1]] for i, j in enumerate(zip(test_lp, test_tagd)) if j[0]!=j[1]]

In [ ]:
incorrect_tagged_cases

In [ ]:
incorrect_tagged_cases_lp

In [ ]:
#checking accuracy
print('accuracy of vanila viterbi form = ',accuracy)
print('accuracy of viterbi + laplace smoothing form = ',accuracy_lp)

## Viterbi + ruled base tagging to deal with unknown words

In [ ]:
# defining regexp for ruled base tagging

patterns = [
    (r'.*ing$', 'VERB'),              
    (r'.*ed$', 'VERB'),               
    (r'.*es$', 'VERB'),               
    (r'.*ould$', 'VERB'),              
    (r'.*\'s$', 'NOUN'),              
    (r'.*s$', 'NOUN'),                
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'),
    (r'.*able$', 'ADJ'),
    (r'.*ly$', 'ADV'),
    (r'.*', 'NOUN')
      
]

regexp_tagger = nltk.RegexpTagger(patterns)

In [ ]:
# regexp_tagger.tag(['Editorials'])[0][1]

In [ ]:
# Viterbi + Rule based tagger 

def h_viterbi_rb(words,train_bag=train_tagd):
    pos=[]
    T=list(set([i[1] for i in train_bag]))
    wtr=list(set([i[0] for i in train_bag]))
    
    for index, word in enumerate(words):

        if word in wtr:
            state_p=[]
            for tag in T:
                if index ==0:
                    transition_p=tags_df.loc['.',tag]
                else:
                    transition_p = tags_df.loc[pos[-1],tag]

                #emission probability
                emission_p = w_g_t(words[index],tag)
                state=transition_p * emission_p
                state_p.append(state)

            max_p=max(state_p) #evaluating maximum likelyhood

            w_pos=T[state_p.index(max_p)]
            pos.append(w_pos)
        else:
            wordx=[]
            wordx.append(word)
            w_pos=regexp_tagger.tag(wordx)
            pos.append(w_pos[0][1])
            
    
    return list(zip(words,pos))

In [ ]:
#checking accuracy

test_rb = h_viterbi_rb(test_words)
chk_tags = [i for i, j in zip(test_rb, test_tagd) if i == j] 

accuracy_rb = len(chk_tags)/len(test_rb)
accuracy_rb

In [ ]:
# incorrect tag cases for rb
incorrect_tagged_cases_rb = [[test_tagd[i-1],j,test_tagd[i+1]] for i, j in enumerate(zip(test_rb, test_tagd)) if j[0]!=j[1]]

### Accuracy comparison

In [ ]:
# accuracy comparison

print('accuracy of vanila viterbi form = ',accuracy)
print('accuracy of viterbi + laplace smoothing form = ',accuracy_lp)
print('accuracy of viterbi + rule based form = ',accuracy_rb)

#Viterbi + rule based form has high tagging accuracy

## Checking with test samples:

In [ ]:
# checking with test samples

text=open('Test_sentences.txt').read()
text

In [ ]:
# creating tokens
token_txt=word_tokenize(text)
token_txt

In [ ]:
# tagging by vanila viterbi algorithm

st=h_viterbi(token_txt)
st

In [ ]:
# tgging by modified viterbi algorithm (laplace smoothing)
lp=h_viterbi_lp(token_txt)
lp

In [ ]:
#tagging by viterbi + rule base tagger

rb=h_viterbi_rb(token_txt)
rb

In [ ]:
#comparison

#EXTRACTING POS TAGS
st_tags=[i[1] for i in st]
lp_tags=[i[1] for i in lp]
rb_tags=[i[1] for i in rb]


# Many words(such as Android, Google,NASA,Satellite, etc.) which were tagged wrongly by vanila viterbi are corrected by viterbi + rule based tagger

compare=pd.DataFrame({'Words':token_txt,'vanila viterbi':st_tags,'Laplace viterbi':lp_tags,'rule based viterbi':rb_tags})
compare